## Inter-subject correlation

In [ ]:
# Compute interSC 
classical_data = [None] * nCohort

for i in range(nCohort):
    classical_data[i] = np.load(f"{npy_dir}/{cROI}_6mm/{cohort_IDs['subject'][i]}_6mm.npy")

nVoxels = classical_data[0].shape[0]
classical_patterns = np.zeros((nVoxels, 2, nCohort))

for i in range(nCohort):
    classical_patterns[:, 0, i] = np.mean(classical_data[i][:, all_CSminus_onsets[i]], axis = 1)
    classical_patterns[:, 1, i] = np.mean(classical_data[i][:, all_CSplus_onsets[i]], axis = 1)
    
subs = np.arange(0, nCohort)
classical_heldout = np.zeros_like(classical_patterns)
    
for i in subs:
    sel_subs = subs[subs!= i]
    classical_heldout[:, :, i] = np.mean(classical_patterns[:, :, sel_subs], axis = 2)
    
all_mats = np.zeros((2, 2, nCohort))
ROI_interSC = np.zeros((2, nCohort))

for i in range(nCohort):
    sub_mat = np.corrcoef(classical_patterns[:,:,i], classical_heldout[:,:,i], rowvar = False)
    sub_mat = sub_mat[:2, 2:]
    all_mats[:,:,i] = sub_mat
    ROI_interSC[:, i] = np.diagonal(sub_mat)

    
threat_control_isc = np.zeros(nCohort)
threat_threat_isc = np.zeros(nCohort)

for i in range(nCohort):
    threat_control_isc[i] = all_mats[:,:,i][1,0]
    threat_threat_isc[i] = all_mats[:,:,i][1,1]

threat_threat_med = np.median(threat_threat_isc)
threat_control_med = np.median(threat_control_isc)

threat_threat_iqr = iqr(threat_threat_isc)
threat_control_iqr = iqr(threat_control_isc)

## Discrimination

In [ ]:
# Calculate discrimination
diff_scores = np.zeros(nCohort)
for i in range(nCohort):
    diff_scores[i] = threat_threat_isc[i] - threat_control_isc[i] 
    logical = diff_scores > 0
disc_rate = sum(logical)/nCohort * 100

# Generate null distribution
null_dist_disc = np.zeros(nPerms)

for k in range(nPerms):

    all_classical_patterns = np.zeros((nVoxels, 2, nCohort))

    for i in range(nCohort):
        concat_CS = np.asarray(all_CSminus_onsets[i] + all_CSplus_onsets[i])
        break_point = len(all_CSminus_onsets[i])
        concat_CS = concat_CS[random.choices(range(len(concat_CS)), k = (len(concat_CS)))]    
        all_classical_patterns[:, 0, i] = np.mean(classical_data[i][:, concat_CS[:break_point]], axis = 1)
        all_classical_patterns[:, 1, i] = np.mean(classical_data[i][:, concat_CS[break_point:]], axis = 1)

    subs = np.arange(0, nCohort)
    all_classical_heldout = np.zeros_like(all_classical_patterns)

    for i in subs:
        sel_subs = subs[subs!= i]
        all_classical_heldout[:, :, i] = np.mean(all_classical_patterns[:, :, sel_subs], axis = 2)

    all_mats = np.zeros((2, 2, nCohort))

    for i in range(nCohort):
        sub_mat = np.corrcoef(all_classical_patterns[:,:,i], all_classical_heldout[:,:,i], rowvar = False)
        all_mats[:,:,i] = sub_mat[:2, 2:]
        
    # Classification
    diff_scores = np.zeros(nCohort)

    for i in range(nCohort):
        shuff_threat_control_isc = all_mats[:,:,i][1,0]
        shuff_isc_plus = all_mats[:,:,i][1,1]
        diff_scores[i] = shuff_isc_plus - shuff_threat_control_isc 
        logical = diff_scores > 0
    
    null_dist_disc[k] = (sum(logical)/nCohort * 100)
    
disc_p_val = len((null_dist_class[null_dist_class > disc_rate]) + 1) / (nPerms + 1)

## InterSC vs. Symptom Severity

In [ ]:
# Define function that correlates an interSC variable with each dimension of a symptom variable
def corr_isc_symp(interSC_var, symp_var): 
    actual_corrs = [None] * len(symptom_labels)
    corr_p_vals = [None] * len(symptom_labels)
    all_null_dists = [None] * len(symptom_labels)
    
    for i in range(len(symptom_labels)):
        which_score = symptom_labels[i]
        null_dist = np.zeros(nPerms)

        # Actual r value
        x = symp_var[which_score]
        y = interSC_var
        actual_corr = stats.spearmanr(x, y)[0]
        actual_corrs[i] = actual_corr

        # Null dist of r-values 
        for j in range(nPerms):
            x = symp_var[which_score]
            y = interSC_var[random.choices(range(nCohort), k = (nCohort))]
            null_dist[j] = stats.spearmanr(x, y)[0]
    
        perms_above_frac = len((null_dist[null_dist > actual_corr]) + 1) / (nPerms + 1)
        corr_p_vals[i] = min(2*perms_above_frac, 2*(1-perms_above_frac))
        all_null_dists[i] = null_dist
        
    return actual_corrs, corr_p_vals, all_null_dists

In [ ]:
# Correlate interSC with symptom severity
ROI_rho_vals, ROI_p_vals, ROI_nulls =  corr_isc_symp(threat_threat_isc, CAPS_data)

## Final values

In [ ]:
# Assemble final values into list
ROI_values = [threat_threat_med, 
            threat_threat_iqr, 
            threat_control_med, 
            threat_control_iqr, 
            disc_rate, 
            disc_p_val]

ROI_values.extend(ROI_rho_vals)
ROI_values.extend(ROI_p_vals)

ROI_value_labels = ['threat_threat_median', 
                    'threat_threat_iqr', 
                    'threat_control_median', 
                    'threat_control_iqr', 
                    'disc_rate', 
                    'disc_p_val',
                    'rexp_r',
                    'avoid_numb_r',
                    'hyp_r',
                    'rexp_p',
                    'avoid_numb_p',
                    'hyp_p']